In [1]:
%matplotlib inline

In [2]:
from collections import namedtuple
import os, sys
print( os.getcwd() )
print( os.listdir( os.getcwd() ) )
import time

/home/topolo/PropD/MLgrabbag/kaggle
['LSTM_model201702271930.save', 'cleaning_dueSigmaFin.pyc', 'LSTM_model201702280608.save', '.ipynb_checkpoints', 'dueSigmaFinancial_kaggle.py', 'LSTM_model.save', 'glass-classification.zip', 'DueSigmaFin_runs.ipynb', 'dueSigmaFinancial_local.ipynb', 'kaggle.ipynb', 'glass.csv', 'train.h5.zip', '__init__.py', 'train.h5', 'cleaning_dueSigmaFin.py']


In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [4]:
import theano

Using gpu device 0: GeForce GTX 980 Ti (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5105)
/home/topolo/Public/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [5]:
from theano import function, config, sandbox, shared 
import theano.tensor as T

In [6]:
print( theano.config.device )
print( theano.config.lib.cnmem)  # cf. http://deeplearning.net/software/theano/library/config.html
print( theano.config.print_active_device)# Print active device at when the GPU device is initialized.

gpu0
0.8
True


In [7]:
print(theano.config.allow_gc)
print(theano.config.optimizer_excluding)

False



In [8]:
from six.moves import cPickle

In [9]:
sys.path.append( '../ML' )

In [10]:
from LSTM_Right import *

# 2Sigma Financial Time series

In [11]:
timeseries_pd = pd.read_hdf( 'train.h5')

In [12]:
import cleaning_dueSigmaFin
from cleaning_dueSigmaFin import clean_tseries, clean_test, id_only, split_tseries,y_only, calc_R

## Split up given time series, $X(t),y(t)$ into train, valid, test 

In [13]:
train_to_all = 0.85 
T_0 = 906 # timestamp to stop at
num_valid = 10 # how many time steps to validate on
obs_train = timeseries_pd[timeseries_pd["timestamp"]<int(T_0*train_to_all)]  # kaggle call these "observation"

In [14]:
# clean training data
cleaned_obs_train = clean_tseries( obs_train)

In [15]:
obs_valid = timeseries_pd[
    (timeseries_pd['timestamp']>=int(T_0*train_to_all)) & (timeseries_pd['timestamp']<int(T_0*(train_to_all)+num_valid))]

In [16]:
cleaned_obs_valid = clean_tseries( obs_valid)

## "Tuning" parameters

In [18]:
L = Gates(g=3,i=2,f=2,o=2)
n_in = 110 # s1
s_2  = 80  # 80
H = 50

s_l = Gates(g=[n_in,s_2,H],i=[n_in,H],f=[n_in,H],o=[n_in,H])

activations=Psis(g=(T.tanh,None),i=(T.nnet.sigmoid,T.nnet.sigmoid),f=(T.nnet.sigmoid,T.nnet.sigmoid),
                 o=(T.nnet.sigmoid,T.nnet.sigmoid),h=(T.tanh,))

In [17]:
# this is for a second try, with smaller number of layers
L = Gates(g=3,i=2,f=2,o=2)
n_in = 110 # s1
s_2  = 50  # 80
H = 20

s_l = Gates(g=[n_in,s_2,H],i=[n_in,H],f=[n_in,H],o=[n_in,H])

activations=Psis(g=(T.tanh,None),i=(T.nnet.sigmoid,T.nnet.sigmoid),f=(T.nnet.sigmoid,T.nnet.sigmoid),
                 o=(T.nnet.sigmoid,T.nnet.sigmoid),h=(T.tanh,))

In [18]:
lambda_learn =0.9  # learning rate
alpha_reg = 0.01 # regularization constant
beta_mom = 0.0001 # momentum constant, old "Mo" (it's an inside joke)

# Train Deep LSTM Model

In [23]:
# logistic regression/for classification
LSTM_model=LSTM_Model_right(L,s_l,H,1,activations, None )
Memblck = MemoryBlock_right(H, LSTM_model)
Memblck.build_scan_over_t()
Memblck.build_J(lambda_learn)  # inputted value, lambda_val, is the learning rate


Total number of parameters: 19 
Total number of parameters: 19 


Elemwise{add,no_inplace}.0

In [19]:
# for the L2 norm
LSTM_model=LSTM_Model_right(L,s_l,H,1,activations, None )
Memblck = MemoryBlock_right(H, LSTM_model)
Memblck.build_scan_over_t()
Memblck.build_J_L2norm(lambda_learn)  # inputted value, lambda_val, is the learning rate

Total number of parameters: 19 
Total number of parameters: 19 


Elemwise{add,no_inplace}.0

In [20]:
# be aware that this step may take more than a few seconds
Memblck.build_update(alpha=alpha_reg,beta=beta_mom)

Total number of parameters: 19 


Do the actual training; *MANUALLY* change `MAX_ITERS`

In [51]:
MAX_ITERS = 15 # 10
%time results_cost = Memblck.train_model_full( cleaned_obs_train, MAX_ITERS ) # 2
# CPU times: user 11min 50s, sys: 2min 27s, total: 14min 17s
# Wall time: 14min 17s


theano.config.allow_gc =:  False
CPU times: user 1h 29min 23s, sys: 19min 59s, total: 1h 49min 22s
Wall time: 1h 49min 19s


## Predicting with the trained model

In [52]:
results_cost

array([  6.94319834e-310,   6.94319834e-310,   6.94319834e-310,
         6.94319834e-310,   0.00000000e+000,   9.53546696e-322,
         1.12585395e-315,   1.36592013e-315,   6.94304809e-310,
         6.94304678e-310,   2.37151510e-322,   2.37151510e-322,
         1.13517005e-315,   6.94319834e-310,   3.95252517e-322])

In [33]:
obs_valid.describe()

,id,timestamp,derived_0,derived_1,derived_2,derived_3,derived_4,fundamental_0,fundamental_1,fundamental_2,...,technical_36,technical_37,technical_38,technical_39,technical_40,technical_41,technical_42,technical_43,technical_44,y
count,86246.000000,86246.000000,82946.000000,8.273400e+04,65897.000000,79714.000000,65472.000000,85245.000000,52994.000000,67080.000000,...,86214.000000,8.617800e+04,8.617800e+04,8.608700e+04,86214.000000,85930.000000,8.608700e+04,8.608700e+04,83168.000000,86246.000000
mean,1089.804976,768.986353,-0.337620,2.928802e+12,-0.369657,-0.283734,-0.077146,-0.001263,-0.032461,-0.174287,...,-0.056573,-1.128401e-01,-1.215678e-01,-9.838293e-02,0.068058,0.004546,-2.476770e-02,-7.713747e-01,0.000191,0.000235
std,633.683668,26.282748,18.288225,1.119510e+14,17.126638,13.742033,0.946136,0.266492,1.775605,3.361082,...,0.432223,2.746581e-01,2.767393e-01,2.559960e-01,0.306142,0.111926,1.901646e-01,9.396070e-01,0.031364,0.016204
min,0.000000,724.000000,-505.480621,-6.626774e-02,-800.364685,-1028.421143,-18.364044,-0.469985,-58.002731,-49.234299,...,-1.684077,-1.000000e+00,-1.000000e+00,-1.000000e+00,-0.457321,-0.444953,-1.000000e+00,-2.000000e+00,-0.106292,-0.086094
25%,542.000000,746.000000,-0.051800,-2.919330e-02,-0.060873,-0.134821,-0.127265,-0.191126,-0.175051,-0.226237,...,-0.268161,-3.894686e-03,-1.477495e-02,-1.068642e-03,-0.143311,-0.073135,-3.087161e-14,-1.999992e+00,-0.021578,-0.007393
50%,1090.000000,769.000000,0.043903,6.437163e-03,0.015760,0.012506,-0.007520,-0.026204,0.020613,-0.054855,...,-0.024910,-4.303226e-08,-6.466906e-09,-7.904232e-12,-0.010224,0.001104,-4.247640e-31,-1.184154e-02,0.000120,-0.000155
75%,1662.000000,792.000000,0.198591,1.250951e-01,0.202912,0.309584,0.095409,0.154450,0.161489,0.120475,...,0.169945,-5.368916e-19,-1.540634e-19,-3.327274e-23,0.208418,0.075601,0.000000e+00,-9.093837e-13,0.021121,0.007535
max,2156.000000,814.000000,400.364502,4.862839e+15,63.275314,35.819118,7.492327,0.905748,17.744707,76.771248,...,3.058554,0.000000e+00,-0.000000e+00,0.000000e+00,1.569265,0.504141,1.000000e+00,0.000000e+00,0.114493,0.093498


In [24]:
list(y)[0].shape

NameError: name 'y' is not defined

In [53]:
print(len(cleaned_obs_valid));print(type(cleaned_obs_valid[0]));print(type(cleaned_obs_valid[0][0]))
print(cleaned_obs_valid[0][0].shape);print(cleaned_obs_valid[0][0].dtype);
cleaned_obs_valid[0][0].astype(theano.config.floatX).shape

951
<type 'tuple'>
<type 'numpy.ndarray'>
(10, 110)
float32


(10, 110)

In [54]:
# valid_predictions = Memblck.predict_on_lst( cleaned_obs_valid,verbose=False)
valid_predictions = Memblck.predict_on_lst_givens( cleaned_obs_valid,verbose=False)

<<!! BUG IN FGRAPH.REPLACE OR A LISTENER !!>> <type 'exceptions.TypeError'> ('The type of the replacement must be compatible with the type of the original Variable.', AllocEmpty{dtype='float32'}.0, AllocEmpty{dtype='float32'}.0, TensorType(float32, row), TensorType(float32, matrix), 'MergeOptimizer') MergeOptimizer
ERROR (theano.gof.opt): SeqOptimizer apply <theano.gof.opt.EquilibriumOptimizer object at 0x7fcf6c2492d0>
ERROR (theano.gof.opt): Traceback:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/home/topolo/Public/anaconda2/lib/python2.7/site-packages/theano/gof/opt.py", line 230, in apply
    sub_prof = optimizer.optimize(fgraph)
  File "/home/topolo/Public/anaconda2/lib/python2.7/site-packages/theano/gof/opt.py", line 89, in optimize
    ret = self.apply(fgraph, *args, **kwargs)
  File "/home/topolo/Public/anaconda2/lib/python2.7/site-packages/theano/gof/opt.py", line 2223, in apply
    sub_prof = gopt.apply(fgraph)
  File "/home/topolo/Public/anaconda2/lib/p

In [55]:
print(type(valid_predictions));print(len(valid_predictions));print(type(valid_predictions[0]));
print(valid_predictions[0].shape)

<type 'list'>
951
<type 'CudaNdarray'>
(10, 1)


In [56]:
X_cleaned_obs_valid,y_cleaned_obs_valid=split_tseries(cleaned_obs_valid)

In [57]:
print(type(y_cleaned_obs_valid));print(len(y_cleaned_obs_valid));print(type(y_cleaned_obs_valid[0]));
print(y_cleaned_obs_valid[0].shape)

<type 'list'>
951
<type 'numpy.ndarray'>
(10, 1)


In [58]:
np.array( valid_predictions[2] )

array([[ 0.74542069],
       [ 1.63571405],
       [-0.66123116],
       [ 0.01397786],
       [ 0.01397786],
       [ 0.01397786],
       [ 0.01397786],
       [ 0.01397786],
       [ 0.01397786],
       [ 0.10692236]], dtype=float32)

In [59]:
valid_predictions_np = [np.array(arr) for arr in valid_predictions]

In [60]:
#R_valid = calc_R(valid_predictions,y_cleaned_obs_valid)
R_valid = calc_R(valid_predictions_np,y_cleaned_obs_valid)
print(R_valid)


-71.6583193351


In [96]:
tempparams = Memblck.LSTM_model.__get_state__()['params']
print(type(tempparams));print(len(tempparams));print(type(tempparams[0]));print(tempparams[0].get_value().shape)
tempparams[0].get_value();
dir(tempparams[0]);

Total number of parameters: 19 
<type 'list'>
19
<class 'theano.sandbox.cuda.var.CudaNdarraySharedVariable'>
(110, 80)


In [99]:
f = open("LSTM_model.save",'wb')
for param in Memblck.LSTM_model.__get_state__()['params']:
    cPickle.dump( param.get_value(), f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()

Total number of parameters: 19 


In [109]:
print(Memblck.LSTM_model._gates.g.Thetabs);print(Memblck.LSTM_model._gates.g.L);
print(Memblck.LSTM_model._gates.g.Thetabs[0].l);
print(len(Memblck.LSTM_model._gates.g.Thetabs[0].__get_state__()['params']))

[<LSTM_Right.Thetabtheta_right object at 0x7fd951ddda50>, <LSTM_Right.Thetab_right object at 0x7fd9518bbed0>]
3
1
3


### Saving model, outputting it out to paste into exterior code

#### Loading back the values of the last saved model

In [10]:
f = open("LSTM_model.save",'rb')
19
loaded_param = []
for _ in range(19):
    loaded_param.append(cPickle.load(f))

In [12]:
for param in loaded_param:
    print type(param),param.shape

 <type 'numpy.ndarray'> (110, 80)
<type 'numpy.ndarray'> (80,)
<type 'numpy.ndarray'> (50, 80)
<type 'numpy.ndarray'> (80, 50)
<type 'numpy.ndarray'> (50,)
<type 'numpy.ndarray'> (110, 50)
<type 'numpy.ndarray'> (50,)
<type 'numpy.ndarray'> (50, 50)
<type 'numpy.ndarray'> (50, 50)
<type 'numpy.ndarray'> (110, 50)
<type 'numpy.ndarray'> (50,)
<type 'numpy.ndarray'> (50, 50)
<type 'numpy.ndarray'> (50, 50)
<type 'numpy.ndarray'> (110, 50)
<type 'numpy.ndarray'> (50,)
<type 'numpy.ndarray'> (50, 50)
<type 'numpy.ndarray'> (50, 50)
<type 'numpy.ndarray'> (50, 1)
<type 'numpy.ndarray'> (1,)


In [18]:
LSTM_model=LSTM_Model_right(L,s_l,H,1,activations, None )
Memblck = MemoryBlock_right(H, LSTM_model)

In [20]:
Memblck.LSTM_model.__set_state__(*loaded_param)
# Memblck.__set_state__(*loaded_param)


NameError: global name 'Thetab_vals' is not defined

** Bottom Line** for loading back values of last saved model

In [20]:
LSTM_model=LSTM_Model_right(L,s_l,H,1,activations, None )
Memblck = MemoryBlock_right(H, LSTM_model)

In [23]:
# MANUALLY change filename
f = open("LSTM_model.save",'rb')
number_of_Thetabs = len( Memblck.__get_state__() )
loaded_param = []
for _ in range(number_of_Thetabs):
    loaded_param.append(cPickle.load(f))

Total number of parameters: 19 


In [24]:
Memblck.__set_state__(*loaded_param)

Then you can go ahead and further train the model

In [29]:
Memblck.build_scan_over_t()
Memblck.build_J(lambda_learn) 

Total number of parameters: 19 
Total number of parameters: 19 


Elemwise{add,no_inplace}.0

In [30]:
# be aware that this step may take more than a few seconds
Memblck.build_update(alpha=alpha_reg,beta=beta_mom)

Total number of parameters: 19 


In [52]:
MAX_ITERS = 50
%time results_cost = Memblck.train_model_full( cleaned_obs_train, MAX_ITERS ) # MAX_ITERS=30
# CPU times: user 3h 29min 15s, sys: 44min 25s, total: 4h 13min 41s
# Wall time: 4h 13min 35s
# MAX_ITERS=50
# CPU times: user 5h 49min 19s, sys: 1h 16min 13s, total: 7h 5min 33s
# Wall time: 7h 5min 23s

theano.config.allow_gc =:  False
CPU times: user 5h 49min 19s, sys: 1h 16min 13s, total: 7h 5min 33s
Wall time: 7h 5min 23s


In [61]:
time.strftime( "%Y%m%d%H%M",time.gmtime())

'201702282350'

In [62]:
f = open('LSTM_model'+time.strftime('%Y%m%d%H%M',time.gmtime())+'.save' ,'wb')
for param in Memblck.__get_state__():
    cPickle.dump( param, f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()

Total number of parameters: 19 


[How to create a Pandas DataFrame from String](http://stackoverflow.com/questions/22604564/how-to-create-a-pandas-dataframe-from-string)

[Convert a numpy.ndarray to string(or bytes) and convert it back to numpy.ndarray
](http://stackoverflow.com/questions/30167538/convert-a-numpy-ndarray-to-stringor-bytes-and-convert-it-back-to-numpy-ndarray)

In [13]:
import sys
print(sys.version_info[0])
if sys.version_info[0]<3:
    from StringIO import StringIO
else:
    from io import StringIO
# An in-memory stream for unicode text



2


Also, we can use the `tostring` and `fromstring` methods from numpy

In [22]:
loaded_param[0].tostring();

In [25]:
print(loaded_param[0].dtype)

float32


In [31]:
[weight.shape for weight in Memblck.__get_state__()]

Total number of parameters: 19 


[(110, 80),
 (80,),
 (50, 80),
 (80, 50),
 (50,),
 (110, 50),
 (50,),
 (50, 50),
 (50, 50),
 (110, 50),
 (50,),
 (50, 50),
 (50, 50),
 (110, 50),
 (50,),
 (50, 50),
 (50, 50),
 (50, 1),
 (1,)]

## Extraneous copies of Python functions, code

In [28]:
def split_tseries(data_lst):
    X,y=list( zip(*data_lst) )
    X=list(X)
    y=list(y)
    return X,y

In [45]:
def calc_R( yhat,y):
    m = len(y)
    assert m == len(yhat)
    mu = sum([yvals.mean() for yvals in y])/m
#    R= sum( [(y[i]-yhat[i])**2/(y[i]-mu)**2 for i in range(m)] )
    R = [ np.power( (y[i]-yhat[i]),2).sum()/ np.power( (y[i]-mu),2).sum() for i in range(m)]
    R = sum(R)/m
    R = 1 - R
    R = np.sign(R) * np.sqrt( np.abs(R))
    return R

##### Playground

In [110]:
def spit_output(*args):
    a=args[0]
    b=args[1]
    print "This is a:",a
    print "This is b:",b
    return a,b

In [111]:
a1,b2=spit_output(*[3,55])

This is a: 3
This is b: 55


In [113]:
a1,b2=spit_output(5,22)

This is a: 5
This is b: 22


In [119]:
print(len( Memblck.LSTM_model._gates.g.Thetabs[0].__get_state__()['params'] ))
print(len( Memblck.LSTM_model._gates.g.Thetabs[1].__get_state__()['params'] ))
print(range(3));print(range(7))

3
2
[0, 1, 2]
[0, 1, 2, 3, 4, 5, 6]


In [123]:
print(len(Memblck.LSTM_model._gates));
print(len(Memblck.LSTM_model._gates[0].__get_state__()['params']))
Memblck.LSTM_model._gates[0]

4
5


In [23]:
loaded_param[0].tostring()

'\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xd5\xec27\xf4I1\xb7\x12\x00\x00\x00\x12\x00\x00\x80\xe7`17\xd2\x0b27\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00a\x0427\x12\x00\x00\x00\x12\x00\x00\x00\x1b\xf5+\xb7\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80k\r37\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xabV4\xb7\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xcaZ2\xb7\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xc5\x0e4\xb7\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00

In [26]:
ts='\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xd5\xec27\xf4I1\xb7\x12\x00\x00\x00\x12\x00\x00\x80\xe7`17\xd2\x0b27\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00a\x0427\x12\x00\x00\x00\x12\x00\x00\x00\x1b\xf5+\xb7\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80k\r37\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xabV4\xb7\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xcaZ2\xb7\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xc5\x0e4\xb7\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xbc\xef1\xb7\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00/\xee1\xb7\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80J\xa6\xe13\x07\x9f\xde\xb3\x12\x00\x00\x00\x12\x00\x00\x80r}\xe33\x8ce\xe03\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00{\xad\xdf3\x12\x00\x00\x00\x12\x00\x00\x00C\x14\xd7\xb3\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x9c\xd9\xe13\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00f\xa7\xe9\xb3\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xfd\xe2\xe0\xb3\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00K\xa9\xe2\xb3\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xe6#\xe1\xb3\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xd8\xdb\xe0\xb3\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xc5Y\x04\xaf\x06\xac\n/\x12\x00\x00\x80\x12\x00\x00\x00\xceW\x04\xafu\xb9\x04\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80]w\n\xaf\x12\x00\x00\x80\x12\x00\x00\x80\xe8H\x0b/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xfcH\x04\xaf\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xd4#\x05/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00l\x98\x04/\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xc2m\t/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xbd\x9a\x04/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80c\xa7\x04/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x1e\xd5\xbf/\xe7\xba\xbd\xaf\x12\x00\x00\x80\x12\x00\x00\x00C_\xbe/\xd9\xd7\xbe/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80+\x8b\xbe/\x12\x00\x00\x80\x12\x00\x00\x80\n\xd0\xb7\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00H\xfa\xbf/\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x06\xe5\xc1\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xb81\xbf\xaf\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xaa\xd8\xc0\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00/\xca\xbe\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\xc3\xbe\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80(K\xe8\xb0\xc9\x89\xe60\x12\x00\x00\x00\x12\x00\x00\x80\x03\xc0\xe6\xb0o6\xe7\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x17q\xe7\xb0\x12\x00\x00\x00\x12\x00\x00\x00\xa4\xd9\xdf0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80xs\xe8\xb0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00"\xa2\xea0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xd3\x98\xe70\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x9e\xf5\xe90\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x89\'\xe70\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x9b\x1f\xe70\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x17y\xe4/\x12}\xe0\xaf\x12\x00\x00\x80\x12\x00\x00\x00\xd4\x15\xe1/\xcc\x19\xe3/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x80\x96\xe1/\x12\x00\x00\x80\x12\x00\x00\x80\xf0\x8f\xd8\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00u\xab\xe4/\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x84\xb1\xe4\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00s\x91\xe3\xaf\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xf3\xca\xe4\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00h\xb9\xe2\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80(\xc7\xe2\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00+\x0b\xbc3\xd7\x9f\xba\xb3\x12\x00\x00\x80\x12\x00\x00\x00\xda\xcb\xba3\xda/\xbb3\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x1eX\xbb3\x12\x00\x00\x80\x12\x00\x00\x80\x92M\xb5\xb3\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x10+\xbc3\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xaa\xdb\xbd\xb3\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xd0}\xbb\xb3\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80@V\xbd\xb3\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xc1"\xbb\xb3\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xda\x1c\xbb\xb3\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00O\x9dp/m\xe4m\xaf\x12\x00\x00\x80\x12\x00\x00\x00Y\xc1l/\x0eoo/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80v\xdan/\x12\x00\x00\x80\x12\x00\x00\x80\x1e\xcdf\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x1a\xc7p/\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xe2\x9fo\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00d\xd3o\xaf\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80C\x91q\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x003\xe5n\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x87\x01o\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x82KP\xba\xaf\x85\xb86\x12\x00\x00\x00\x12\x00\x00\x80+~\xd2\xbb\x16>_\xba\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x85+\xbb\xb6\x12\x00\x00\x00\x12\x00\x00\x007\x8e\xd48\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x1f\x93T\xba\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x1b/\x15<\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x19\xe3l:\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x15(L7\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xa7c\t;\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x005\x94\xe1:\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xf0\xa3#0\x9a\xb0!\xb0\x12\x00\x00\x80\x12\x00\x00\x00e\xb9!0\xc9\xc8"0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80}c"0\x12\x00\x00\x80\x12\x00\x00\x80\x0c\x9a\x1c\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00X\xc3#0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x91K$\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00E\x14#\xb0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80.a$\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x00\x97"\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80Y\x9c"\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xf7h\xee\xadD\xcd\xe7-\x12\x00\x00\x00\x12\x00\x00\x80\x0c?\xe9\xad\x10\x16\xec\xad\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00J\x96\xe9\xad\x12\x00\x00\x00\x12\x00\x00\x00\xe2\xbf\xdb-\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x95\xc0\xee\xad\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x96\xc9\xef-\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80f\xe0\xec-\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x07\xfe\xee-\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80{\x93\xeb-\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00D\xa1\xeb-\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xbeSE1c~C\xb1\x12\x00\x00\x00\x12\x00\x00\x80\xde\xa8C1Y\\D1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x8dKD1\x12\x00\x00\x00\x12\x00\x00\x00<\xa0=\xb1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x93wE1\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00|\xe7F\xb1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80F\xb3D\xb1\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x8aF\xb1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80<?D\xb1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xfd<D\xb1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00+\t\x9f/\xa1\xf2\x9c\xaf\x12\x00\x00\x80\x12\x00\x00\x00+)\x9d/\xe6\x16\x9e/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x96\xb6\x9d/\x12\x00\x00\x80\x12\x00\x00\x80#x\x97\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00T,\x9f/\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80)\x19\xa0\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xcdj\x9e\xaf\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80c\xe6\x9f\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xca\xec\x9d\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xd5\xee\x9d\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xf7\xfc\x8f/\x98\xc8X\xaf\x12\x00\x00\x80\x12\x00\x00\x00D\xc0E0C+\x92/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x19sY/\x12\x00\x00\x80\x12\x00\x00\x80(;\\\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xa1\xc5\x90/\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80l:\x81\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x05\xa2\x94\xaf\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80,\xce[\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\n\xe1\xc9\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x0e\x10\xb9\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x83\xeb\x92/K8\x91\xaf\x12\x00\x00\x80\x12\x00\x00\x00ko\x91/\x8b\x1c\x92/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x809\xe1\x91/\x12\x00\x00\x80\x12\x00\x00\x80Ly\x8c\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x95\t\x93/\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xb6\x0c\x94\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xbfd\x92\xaf\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80P\xbf\x93\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00+\xff\x91\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xde\xfe\x91\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00P\xe5p\xaf\x19\xafn/\x12\x00\x00\x80\x12\x00\x00\x00\x17\x9bn\xaf`\xc0o\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x19\xa2o\xaf\x12\x00\x00\x80\x12\x00\x00\x80\xb3\xadg/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00^\x0fq\xaf\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x94Fr/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00n&p/\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xaeLr/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x9a\x8co/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x801\x8fo/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\t\xb5\x0b0\x99s\n\xb0\x12\x00\x00\x80\x12\x00\x00\x00\xd2\xbe\n0\xca\x08\x0b0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80T\x03\x0b0\x12\x00\x00\x80\x12\x00\x00\x80#U\x06\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00,\xce\x0b0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xb1\x17\r\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xf8E\x0b\xb0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xe2\x95\x0c\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x9f\xff\n\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xae\xfa\n\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x83\xbe\xfd-\'G\xeb\xad\x12\x00\x00\x80\x12\x00\x00\x00y\xee\xe9-\xf3,\xfa-\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xdf\xd2\xed-\x12\x00\x00\x80\x12\x00\x00\x80}!\xda\xad\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xf09\xfe-\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80%\x07\xec\xad\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x17=\xfb\xad\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x1b&\xf6\xad\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00&\xd8\xf6\xad\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xd2\xb3\xf7\xad\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00D\x8b#1\x04\xc9 \xb1\x12\x00\x00\x80\x12\x00\x00\x00\t3!1 ^"1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xfa\xb7!1\x12\x00\x00\x80\x12\x00\x00\x80<K\x1a\xb1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00M\xb7#1\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x90\xc9$\xb1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xec\xc5"\xb1\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x94m$\xb1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x19)"\xb1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xf6+"\xb1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xf0\xd4\x970U\x1f\x96\xb0\x12\x00\x00\x80\x12\x00\x00\x00\xcbO\x960.\x07\x970\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xd7\xc7\x960\x12\x00\x00\x80\x12\x00\x00\x80uZ\x91\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xbe\xf2\x970\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x1d\xe7\x98\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xedN\x97\xb0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80^\xa5\x98\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xea\xe7\x96\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80<\xe8\x96\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00,\xf2\x820\x05\xee\x81\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x87\x07\x820\xecY\x820\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xe9m\x820\x12\x00\x00\x80\x12\x00\x00\x80Dx|\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00<\x08\x830\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80(\x1c\x84\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xe2\x8f\x82\xb0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xa7\xcf\x83\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xf0M\x82\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xb9J\x82\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00z\x1e\xe20\xb1\xcb\xdf\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x15\x01\xe00\xe7\xfa\xe00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x17\xbc\xe00\x12\x00\x00\x80\x12\x00\x00\x80\x0c\xf1\xd8\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xa3H\xe20\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x9e\xb6\xe3\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xd0`\xe1\xb0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xbea\xe3\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x8b\xd0\xe0\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\\\xd0\xe0\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00N:\x08\xb0\xbe#\x070\x12\x00\x00\x80\x12\x00\x00\x00\xca6\x07\xb0$\x97\x07\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80:\xac\x07\xb0\x12\x00\x00\x80\x12\x00\x00\x80\xea5\x030\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xf7Q\x08\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x83h\t0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xc8\xd0\x070\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x1a(\t0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00Q\x88\x070\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80y\x85\x070\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xc5\xb1\xdc0r\x1c\xda\xb0\x12\x00\x00\x80\x12\x00\x00\x00\xd3\x84\xda0\x06\x8c\xdb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x83\r\xdb0\x12\x00\x00\x80\x12\x00\x00\x80\x0fF\xd3\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00G\xdc\xdc0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xbe*\xde\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00~\xf2\xdb\xb0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xb1\xb7\xdd\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00I_\xdb\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xc4_\xdb\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xa4\x90g0\x95<e\xb0\x12\x00\x00\x00\x12\x00\x00\x80+ge0\xffef0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x9a2f0\x12\x00\x00\x00\x12\x00\x00\x00\xf98^\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xd5\xbbg0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xe9Ei\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80o\xcef\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00<\xe7h\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x803;f\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xd6:f\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x89\x17\xfc\xac\x8f\xe5!-\x12\x00\x00\x80\x12\x00\x00\x00\n\xeaa\xac\x9c\xa1\x03\xad\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x1aY\x1d\xad\x12\x00\x00\x80\x12\x00\x00\x80\xd22<-\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xa2\xa6\xf9\xac\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xdb*\xbd(\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x1e\xb4\x00-\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x1b\xd2\x0e-\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00-\xf1\xe4,\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80t\xd2\xef,\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x92\x87\xa6\xae\x0c\xea\x8e.\x12\x00\x00\x00\x12\x00\x00\x80+\xb7\x96\xaeh\x9b\x9f\xae\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x99\xf3\x93\xae\x12\x00\x00\x00\x12\x00\x00\x00\x8d&_.\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80.\x91\xa7\xae\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00:\xaf\xa8.\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xe8\xed\xa1.\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xc4\xe8\xa3.\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80z\xfe\x9d.\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xa90\x9e.\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80K\xaf\xd0/\xe4O\xce\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x1dT\xce/\x00\x9d\xcf/\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xd00\xcf/\x12\x00\x00\x00\x12\x00\x00\x00\xbb\xe9\xc7\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x94\xd6\xd0/\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x1b\x8b\xd1\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xa5\xfb\xcf\xaf\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\r\xad\xd1\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xa7`\xcf\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xbef\xcf\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xde\xac\xbe/9\xaf\xbc\xaf\x12\x00\x00\x00\x12\x00\x00\x80\xae\xe9\xbc/\x91\xae\xbd/\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00;\x80\xbd/\x12\x00\x00\x00\x12\x00\x00\x00P\xc1\xb6\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xc8\xd1\xbe/\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x1e<\xc0\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xa8\x07\xbe\xaf\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x8b\xcc\xbf\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80T\x8e\xbd\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xc1\x8c\xbd\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80Xd\x074._\x06\xb4\x12\x00\x00\x00\x12\x00\x00\x80\xda~\x064\xd3\xc6\x064\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x8c\xe3\x064\x12\x00\x00\x00\x12\x00\x00\x00(\x8c\x02\xb4\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80;{\x074\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x97\xb1\x08\xb4\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xd6\xfe\x06\xb4\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xf4Q\x08\xb4\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80e\xbd\x06\xb4\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00-\xb9\x06\xb4\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xb8\x85\xa7\xae\x87\r\xa3.\x12\x00\x00\x00\x12\x00\x00\x80P\x1b\xa4\xaeur\xa6\xae\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xfc\xe6\xa3\xae\x12\x00\x00\x00\x12\x00\x00\x00o\xff\x9c.\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80w\xac\xa7\xae\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x04\xe6\xa5.\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80O\xcd\xa6.\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x92s\xa6.\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80j\xfb\xa5.\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00v\x13\xa6.\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80BV\xe0/\x08I\xdd\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x91\xd8\xdd/^\xf8\xde/\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xadb\xde/\x12\x00\x00\x00\x12\x00\x00\x00\x1d}\xd5\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80u\x89\xe0/\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x9cd\xe2\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80`r\xdf\xaf\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xcc\x88\xe1\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x08\xca\xde\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x9f\xc8\xde\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x00\x10F/\xe1\x94E\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x8c\x1aE/4,E/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xdaUF/\x12\x00\x00\x80\x12\x00\x00\x80\x1f\xee?\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00[1F/\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80"\xafH\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x008~E\xaf\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x1bnH\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00g/E\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x804$E\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80pL`1ul^\xb1\x12\x00\x00\x00\x12\x00\x00\x80\x91:^18B_1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xb1J_1\x12\x00\x00\x00\x12\x00\x00\x00\x03\x04X\xb1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x8cr`1\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00u\x94a\xb1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x05\x9f_\xb1\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x19\xb2a\xb1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xfe\x12_\xb1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00a\x15_\xb1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xa7\x17\x100dy\x0e\xb0\x12\x00\x00\x00\x12\x00\x00\x80i\xa9\x0e0\xa8U\x0f0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x83\x18\x0f0\x12\x00\x00\x00\x12\x00\x00\x00\x9e\xf8\t\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xbd3\x100\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xc6\x1c\x11\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80S\x99\x0f\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xab\xd9\x10\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80X8\x0f\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x978\x0f\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80-\xbaP0\xb7\xacN\xb0\x12\x00\x00\x00\x12\x00\x00\x80g\xddN0\x06\xa9O0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xe7\x8dO0\x12\x00\x00\x00\x12\x00\x00\x00\x82GH\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xd4\xe1P0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00`sR\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xda\x08P\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x82\x06R\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xfc\x87O\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xd2\x85O\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00i\xc2\xc8/L\x88\xc6\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x81\xe4\xc6/\xef\xba\xc7/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80La\xc7/\x12\x00\x00\x80\x12\x00\x00\x80\x9f^\xc0\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x95\xe8\xc8/\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80T0\xca\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x16\x17\xc8\xaf\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xb7\xc5\xc9\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xf7\x96\xc7\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x1f\x96\xc7\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x8081>0\xd5\xfe;\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x14?<0\x92+=0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xc1\xd4<0\x12\x00\x00\x00\x12\x00\x00\x00#\xf45\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x1fW>0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xe5\x88?\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xbc\x86=\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x983?\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80!\x03=\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xc1\x03=\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80"\x81U1b\xe5P\xb1\x12\x00\x00\x00\x12\x00\x00\x804\xcfQ1t\xa8S1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x8fWR1\x12\x00\x00\x00\x12\x00\x00\x00\x01\x12G\xb1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xa5\xc6U1\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x14HW\xb1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xbdJT\xb1\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00y\x8bV\xb1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xe7SS\xb1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xe7XS\xb1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x95\xf5\x9a.b\xa9\x99\xae\x12\x00\x00\x00\x12\x00\x00\x80_\xc5\x9a.\xd9\xa0\x9a.\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x8f\xfe\x99.\x12\x00\x00\x00\x12\x00\x00\x00\xb2\xf5\x96\xae\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x807\x01\x9b.\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x93j\x9b\xae\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x9c\xc0\x9a\xae\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00W\xb3\x9a\xae\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\t\xa9\x9a\xae\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x85\xa2\x9a\xae\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xfad\x1b1\x02\x19\x1a\xb1\x12\x00\x00\x80\x12\x00\x00\x00\x01C\x1a1m\xab\x1a1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\'\xb4\x1a1\x12\x00\x00\x80\x12\x00\x00\x80\x89\xa1\x15\xb1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xe7\x7f\x1b1\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80J\xc8\x1c\xb1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x14\xed\x1a\xb1\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x08d\x1c\xb1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00*\x9c\x1a\xb1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80u\x98\x1a\xb1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x801\x1d\x940^\xfa\x92\xb0\x12\x00\x00\x00\x12\x00\x00\x80\\\t\x930\x17s\x930\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00[\x89\x930\x12\x00\x00\x00\x12\x00\x00\x00\xbe\xd5\x8e\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xc65\x940\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00wT\x95\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80:\xaf\x93\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x1e\x15\x95\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xb6b\x93\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x04`\x93\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xf2t\x05\xaf\xdf\xa2\x08/\x12\x00\x00\x00\x12\x00\x00\x80A#\x06\xaf\xd9\xdd\x05\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x0ei\x08\xaf\x12\x00\x00\x00\x12\x00\x00\x0009\t/\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x84c\x05\xaf\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xb7\xca\x05/\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xc3\xbb\x05/\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00I\x83\x07/\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80{\xe6\x05/\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xb4\xe7\x05/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xe8\xea\x130\x12V\x12\xb0\x12\x00\x00\x80\x12\x00\x00\x00;\x85\x120\xaa$\x130\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x0b\xf9\x120\x12\x00\x00\x80\x12\x00\x00\x80`\xb9\r\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xa9\x07\x140\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xb2\t\x15\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xfci\x13\xb0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80<\xc4\x14\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xdc\t\x13\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\'\t\x13\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00b\xf6\x10/\x00\xe7\x0e\xaf\x12\x00\x00\x80\x12\x00\x00\x00\xdf\xb5\x0e/\x10\xd9\x0f/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xba\xc7\x0f/\x12\x00\x00\x80\x12\x00\x00\x80E\xbd\x08\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00( \x11/\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xe3O\x12\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x1d;\x10\xaf\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80|Z\x12\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00>\xa4\x0f\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xb9\xa7\x0f\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xb1\xf1<0\x02\x91:\xb0\x12\x00\x00\x80\x12\x00\x00\x00Y\xf1:0\x95\xd5;0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xd2w;0\x12\x00\x00\x80\x12\x00\x00\x80n&4\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x1f\x1b=0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xcd\x81>\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xa18<\xb0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xd1\x07>\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x007\xaf;\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80A\xae;\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x8c\xe5\x100\x94\xba\x0f\xb0\x12\x00\x00\x00\x12\x00\x00\x80e\x9f\x0f0@9\x100\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00JJ\x100\x12\x00\x00\x00\x12\x00\x00\x00\x82\x95\x0b\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80M\xfe\x100\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xd1\xda\x11\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x83u\x10\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00J\xd9\x11\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80m\x1e\x10\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xd3\x1e\x10\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80z\x8876\xb1\x936\xb6\x12\x00\x00\x80\x12\x00\x00\x80L=66\x8d\xd566\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x04.76\x12\x00\x00\x00\x12\x00\x00\x00=\xf91\xb6\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xd1\xa176\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xe8\x9f8\xb6\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xa5\x147\xb6\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xa9\xcc8\xb6\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xd1\xba6\xb6\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xc7\xba6\xb6\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xc1\xb9\x0f5Z`\x0f\xb5\x12\x00\x00\x80\x12\x00\x00\x00\xb2\xbe\x0e5\xf33\x0f5\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80f\xd5\x0f5\x12\x00\x00\x80\x12\x00\x00\x80\xc1\xdc\x0b\xb5\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x8c\xcc\x0f5\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80)\x85\x10\xb5\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x06c\x0f\xb5\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xe9\t\x11\xb5\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xb2\x1d\x0f\xb5\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80Q\x1e\x0f\xb5\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xea\x0e\xe6/GX\xe4\xaf\x12\x00\x00\x80\x12\x00\x00\x00\xdd\xa4\xe4/@\x05\xe5/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xf67\xe5/\x12\x00\x00\x80\x12\x00\x00\x80}\xe0\xdd\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x9b5\xe6/\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80Hg\xe8\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x06d\xe5\xaf\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xab\xa2\xe7\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xcd\xfa\xe4\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x02\xf2\xe4\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00d3R0\xe8\x14P\xb0\x12\x00\x00\x80\x12\x00\x00\x00V\\P0% Q0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xef\xf7P0\x12\x00\x00\x80\x12\x00\x00\x80\xfc\xa2I\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00`[R0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xa2\xf3S\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xce\x80Q\xb0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x801uS\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xa3\x00Q\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xea\xfdP\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x0e\xf6\xaa\xb0\xaf\xc1\xa90\x12\x00\x00\x80\x12\x00\x00\x00\xf4\x9e\xa9\xb0F6\xaa\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80Sc\xaa\xb0\x12\x00\x00\x80\x12\x00\x00\x80*\x0c\xa50\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x97\x11\xab\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80+(\xac0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xb5y\xaa0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xf6 \xac0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00n\x1c\xaa0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x9d\x1b\xaa0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00N\xaa\x120m-\x11\xb0\x12\x00\x00\x80\x12\x00\x00\x00yZ\x110$\xeb\x110\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80&\xcb\x110\x12\x00\x00\x80\x12\x00\x00\x80\xb4\xb2\x0c\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x05\xc6\x120\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80|\xd3\x13\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00+.\x12\xb0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xb3\x85\x13\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00e\xd3\x11\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x11\xd2\x11\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80&\x82\x92\xaemx\x90.\x12\x00\x00\x00\x12\x00\x00\x80l|\x90\xae\xc2\xa1\x91\xae\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x15-\x91\xae\x12\x00\x00\x00\x12\x00\x00\x00\x0bn\x8b.\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80n\xa2\x92\xae\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x04\xfa\x92.\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x80\xee\x91.\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x0077\x93.\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xc7j\x91.\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x88q\x91.\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x800\xa3\n\xaf\x00z\x0b/\x12\x00\x00\x00\x12\x00\x00\x80\x1c\x06\n\xaf+Y\n\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xd8\xc1\x0b\xaf\x12\x00\x00\x00\x12\x00\x00\x00,\xf9\x08/\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xf1\xac\n\xaf\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xfa\x84\x0b/\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xd3s\n/\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x9d}\x0c/\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x88H\n/\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xefI\n/\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80.n00\x82\xec.\xb0\x12\x00\x00\x00\x12\x00\x00\x80\xf9\x05/0^\x8e/0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00O\xa6/0\x12\x00\x00\x00\x12\x00\x00\x00\xd4\xa2)\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xb9\x8e00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00S\n2\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80H\xdd/\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00>\xab1\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80Qy/\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xb0u/\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00q\xffp3\x81-o\xb3\x12\x00\x00\x80\x12\x00\x00\x009fo3\xd9\xe6o3\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80?\x19p3\x12\x00\x00\x80\x12\x00\x00\x80\xf2]h\xb3\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x004(q3\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x802Ps\xb3\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x98Jp\xb3\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xf1\xa5r\xb3\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xfb\xd5o\xb3\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80v\xceo\xb3\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xe4\x08\xa8\xb0\xc9\xcc\xa60\x12\x00\x00\x80\x12\x00\x00\x00\xaa\xf1\xa6\xb0IK\xa7\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xafl\xa7\xb0\x12\x00\x00\x80\x12\x00\x00\x80N&\xa20\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00c$\xa8\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xf7\x93\xa90\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xb7\x8e\xa70\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\\(\xa90\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00X?\xa70\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80m:\xa70\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xddi\x110\xe0\xc6\x0f\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x8e\x14\x100<\xa1\x100\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00pk\x100\x12\x00\x00\x00\x12\x00\x00\x00b$\x0b\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x19\x87\x110\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xf5\xa6\x12\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x96\xe7\x10\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xde<\x12\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xb3\x8a\x10\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x9a\x88\x10\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00B\x12\x120\xba\xa0\x10\xb0\x12\x00\x00\x80\x12\x00\x00\x00B\x99\x100U[\x110\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xd17\x110\x12\x00\x00\x80\x12\x00\x00\x80\x87K\x0c\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00},\x120\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xd3\xd9\x12\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xd8\x9a\x11\xb0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x07\xe0\x12\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00p8\x11\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xd6:\x11\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xd9F-0L\xa4+\xb0\x12\x00\x00\x00\x12\x00\x00\x80\xa2\xdd+0\xc2n,0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xd1W,0\x12\x00\x00\x00\x12\x00\x00\x00\x97\x82&\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x002f-0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x8d\xb3.\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xd7\xba,\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x1aM.\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xccW,\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x15U,\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00c\xaf\xc7/\x91[\xc5\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x8ai\xc5/>\xa8\xc6/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xdf2\xc6/\x12\x00\x00\x80\x12\x00\x00\x80\xe88\xbf\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x17\xd5\xc7/\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x91\x84\xc8\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x03\x03\xc7\xaf\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xe6\x96\xc8\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xefm\xc6\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xe6s\xc6\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xcb\xa3t0E6r\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x85Qr0\xc0ns0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x955s0\x12\x00\x00\x80\x12\x00\x00\x80\x04\xe8j\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00V\xd0t0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xc37v\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x92\xdas\xb0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x08\x04v\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00H=s\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80w>s\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xd1\xf7\xd0/K\x8b\xcf\xaf\x12\x00\x00\x80\x12\x00\x00\x005\xe7\xcf/4\x03\xd0/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xccZ\xd0/\x12\x00\x00\x80\x12\x00\x00\x80\x9e\x88\xc9\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xa5\x1b\xd1/\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80#L\xd3\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x01[\xd0\xaf\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xef\x9b\xd2\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xfd\x04\xd0\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80p\xf9\xcf\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x16\xb7\x1c\xad\x05\xd43-\x12\x00\x00\x00\x12\x00\x00\x80\xe4\xae\xc3\xacs\xde\x1e\xad\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x04\xcb1\xad\x12\x00\x00\x00\x12\x00\x00\x00s\xd5>-\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x07\x1a\x1c\xad\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xe8\x8a\x83,\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80SX\x1d-\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00G\xc1*-\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xc0/\x11-\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xd8l\x15-\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x0b\xea\x14\xaf\xe6\xe9\x13/\x12\x00\x00\x80\x12\x00\x00\x00\xa1\x02\x14\xaf\xfa8\x14\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\t\x7f\x14\xaf\x12\x00\x00\x80\x12\x00\x00\x80\t\x99\x0f/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xe3\x03\x15\xaf\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80{\x84\x16/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00%x\x14/\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80q\x1a\x16/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00m3\x14/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80%-\x14/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00s&\xfd.\xfeg\xfc\xae\x12\x00\x00\x80\x12\x00\x00\x00\xa0\xf4\xfc.aF\xfc.\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80O/\xfd.\x12\x00\x00\x80\x12\x00\x00\x80\x80i\xf6\xae\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x8bG\xfd.\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80>f\x00\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00(\x9a\xfc\xae\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80p?\xff\xae\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xf1s\xfc\xae\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xc5[\xfc\xae\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00R\x1a~0W\x893\xaf\x12\x00\x00\x80\x12\x00\x00\x00\xe5\x18^3P\x03+1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x0124/\x12\x00\x00\x80\x12\x00\x00\x80\x1d\x01\x84\xb5\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x16\x83\xb70\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x99\xd8\xcb\xb3\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x003t\x7f\xb1\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80Q\xfe5\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\tu\xbe\xb2\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xe1N\x98\xb2\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\n\xe0\x05/\x81\xda\x02\xaf\x12\x00\x00\x80\x12\x00\x00\x00\xc4\xef\x03/\xcb#\x05/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80Hr\x03/\x12\x00\x00\x80\x12\x00\x00\x801!\xfd\xae\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xca\xfa\x05/\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x7fz\x05\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xa4c\x05\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x9b-\x05\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xfc\xe9\x04\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80>\xf3\x04\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xb2\xe8\x8e/\x15\xcc\x8d\xaf\x12\x00\x00\x00\x12\x00\x00\x80\xf9\xff\x8d/2I\x8e/\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00QS\x8e/\x12\x00\x00\x00\x12\x00\x00\x00Y\xde\x89\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xc9\xff\x8e/\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xd2\x1e\x90\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xea\x81\x8e\xaf\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xe5\xc8\x8f\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x808?\x8e\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x10;\x8e\xaf\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xb7j\x0b\xb0\x8c\'\n0\x12\x00\x00\x00\x12\x00\x00\x00\xa1!\n\xb0\xb3\xbb\n\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80b\xb8\n\xb0\x12\x00\x00\x00\x12\x00\x00\x00k\x03\x060\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xf2\x83\x0b\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x89e\x0c0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xdd\xf8\n0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00nM\x0c0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x000\xa1\n0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80h\xa1\n0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xc3N\xc7,K{\xc5\xac\x12\x00\x00\x00\x12\x00\x00\x80\xaa\x96\xc5,\xd2R\xc6,\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xebK\xc6,\x12\x00\x00\x00\x12\x00\x00\x00\xb6\x85\xbf\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80Cs\xc7,\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00d\xe9\xc8\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80D\xab\xc6\xac\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00N\x94\xc8\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x114\xc6\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x1e2\xc6\xac\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xef\x02\x06\xb2$\xbd\x042\x12\x00\x00\x80\x12\x00\x00\x00\x11\xbb\x04\xb2,U\x05\xb2\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80XL\x05\xb2\x12\x00\x00\x80\x12\x00\x00\x80C\xa9\x002\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x0b\x1c\x06\xb2\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xbb\xf7\x062\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xd2\x91\x052\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xad\xde\x062\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x9d:\x052\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xe9:\x052\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xcfD.-\x10\xac,\xad\x12\x00\x00\x00\x12\x00\x00\x80\x00\xc4,-\x84h--\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00zb--\x12\x00\x00\x00\x12\x00\x00\x00\x1av\'\xad\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xb7d.-\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xd8\xab/\xad\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xda\xb5-\xad\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00pa/\xad\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x9fM-\xad\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xecK-\xad\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xd8t\x7f,\xaa\x1d}\xac\x12\x00\x00\x00\x12\x00\x00\x80\xbe@},\xe51~,\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x11)~,\x12\x00\x00\x00\x12\x00\x00\x00Kzu\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x9d\xa3\x7f,\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x93\xc1\x80\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80I\xa3~\xac\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\n\x8b\x80\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80~\n~\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x01\x08~\xac\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00< \x02\xb2:\xf9\x002\x12\x00\x00\x80\x12\x00\x00\x00\x96\xed\x00\xb2w|\x01\xb2\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xbd\x80\x01\xb2\x12\x00\x00\x80\x12\x00\x00\x80\xa03\xfa1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xde7\x02\xb2\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80L\x0f\x032\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xb5\xb5\x012\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xd7\xfb\x022\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xe3c\x012\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x03d\x012\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xc4\x91g\xb0\x88Te0\x12\x00\x00\x80\x12\x00\x00\x00 Ne\xb0[af\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\tOf\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x077^0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xbd\xbdg\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80V1i0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00z\xcbf0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x0b\x10i0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x891f0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80s2f0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x81\xf1\xf9\xae\x00\xdc\xf7.\x12\x00\x00\x80\x12\x00\x00\x00\x8f\xac\xf7\xae\x99\xbd\xf8\xae\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xc7\xdb\xf8\xae\x12\x00\x00\x00\x12\x00\x00\x00F\x86\xf0.\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xd1\x1d\xfa\xae\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x801\xa6\xfb.\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00/)\xf9.\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x11\xa5\xfb.\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x0c\x8e\xf8.\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xab\x8e\xf8.\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00t\x13\xaf\xb1\x89{\xad1\x12\x00\x00\x80\x12\x00\x00\x00\xc8r\xad\xb1\xc38\xae\xb1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xbe0\xae\xb1\x12\x00\x00\x80\x12\x00\x00\x80\x9aK\xa81\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xf52\xaf\xb1\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xf5;\xb01\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x1a\x85\xae1\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80w,\xb01\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00(\x16\xae1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xdc\x16\xae1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xc6\xf2\xe8\xb1\x06\xfc\xe61\x12\x00\x00\x80\x12\x00\x00\x00\x10\xd2\xe6\xb1\xd2\xcd\xe7\xb1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x8d\xee\xe7\xb1\x12\x00\x00\x80\x12\x00\x00\x80]\r\xe01\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x07\x1d\xe9\xb1\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xdc\xaa\xea1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00@4\xe81\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x13\x94\xea1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x85\xa2\xe71\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x8b\xa2\xe71\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xc6\xc9{\xb0\xa3Ty0\x12\x00\x00\x80\x12\x00\x00\x80\xb4ez\xb0h\x8ez\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x86[z\xb0\x12\x00\x00\x80\x12\x00\x00\x80\xde\xd1q0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00T\xf8{\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x08\xf1~0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xb0\xffz0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x1e;}0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xaa\x93z0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xf2\x83z0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x97h},\xed\x12{\xac\x12\x00\x00\x00\x12\x00\x00\x80$\xfbz, \x11|,\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xe7,|,\x12\x00\x00\x00\x12\x00\x00\x00\n\x1bs\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80n\x9a},\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x19\x80\x7f\xac\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80*\x89|\xac\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x19G\x7f\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80#\xe2{\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x18\xe1{\xac\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x8fb\xf1\xb1\x19G\xef1\x12\x00\x00\x80\x12\x00\x00\x00\\3\xef\xb1r:\xf0\xb1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xfa<\xf0\xb1\x12\x00\x00\x80\x12\x00\x00\x80o9\xe81\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00.\x8d\xf1\xb1\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80*\x06\xf31\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xf3\xa1\xf01\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x8a\xea\xf21\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xea\x0c\xf01\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80u\r\xf01\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x9c\xee\x04\xae%\xbc\x03.\x12\x00\x00\x80\x12\x00\x00\x80}\xb5\x03\xae\x0fH\x04\xae\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x04F\x04\xae\x12\x00\x00\x00\x12\x00\x00\x80\xde\x94\xff-\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xa0\x06\x05\xae\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xbd\xdd\x05.\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00E\x82\x04.\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xcf\xc7\x05.\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xd7.\x04.\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x0b/\x04.\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x98\xe1\xf0\xb1\xa3\x95\xee1\x12\x00\x00\x80\x12\x00\x00\x00\x8b\x9a\xee\xb1\x9c\xad\xef\xb1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xd8\x93\xef\xb1\x12\x00\x00\x80\x12\x00\x00\x80\xfbU\xe71\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\t\x0e\xf1\xb1\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xb3\x8d\xf21\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00%\x19\xf01\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xc8\\\xf21\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00=~\xef1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xdb~\xef1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x804r@\xaf0\xaf>/\x12\x00\x00\x80\x12\x00\x00\x80p\xa9>\xaf\x99\x7f?\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xdew?\xaf\x12\x00\x00\x80\x12\x00\x00\x80\xdd\xf38/\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x800\x95@\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\t\xccA/\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80_\xd4?/\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xa7\xa8A/\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xb6Z?/\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x0c[?/\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xfb\xdb\x9809t\x97\xb0\x12\x00\x00\x00\x12\x00\x00\x00w\xe6\x970\x9e!\x980\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xdd\x0f\x980\x12\x00\x00\x80\x12\x00\x00\x00#\xfd\x92\xb0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80H\xf7\x980\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xbf\x84\x9a\xb0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\'d\x98\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00=\xc1\x99\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xd1\x1c\x98\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xe8\x15\x98\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x8a}\xac-\xba\xdc\xaa\xad\x12\x00\x00\x00\x12\x00\x00\x80\xa6\xf6\xaa-\x19\xa9\xab-\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xfc\x8c\xab-\x12\x00\x00\x00\x12\x00\x00\x00\x98\xcc\xa5\xad\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x800\x9c\xac-\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xea\xb2\xad\xad\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x80\xf3\xab\xad\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xdcz\xad\xad\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xf7\x8a\xab\xad\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x98\x8a\xab\xad\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xe2\x13\xcd/%r\xca\xaf\x12\x00\x00\x00\x12\x00\x00\x80\xee\x14\xcc/$\x04\xcc/\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xa0S\xcb/\x12\x00\x00\x00\x12\x00\x00\x00c\x06\xc4\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\\<\xcd/\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xca\xda\xcf\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80Af\xcc\xaf\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x93\xd9\xcd\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xc7\x14\xcc\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xa2\x03\xcc\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xe6\x1e\x92\xb0\x94\xca\x900\x12\x00\x00\x80\x12\x00\x00\x00}\xcc\x90\xb0k^\x91\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x801i\x91\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x9cI\x8c0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xd0:\x92\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x93W\x930\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xde\xa1\x910\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x807(\x930\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00EF\x910\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xfeD\x910\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80y+h-\xed\nf\xad\x12\x00\x00\x00\x12\x00\x00\x80\xce*f-\xfc\x05g-\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xf4\xfdf-\x12\x00\x00\x00\x12\x00\x00\x00\xc8\x19_\xad\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xfcUh-\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xcb\tj\xad\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x07mg\xad\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xaa\xa6i\xad\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80,\xe2f\xad\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xe7\xdff\xad\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80Z\xf9b,\xfd\xe4`\xac\x12\x00\x00\x00\x12\x00\x00\x80&\x04a,o\xdaa,\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x91\xd2a,\x12\x00\x00\x00\x12\x00\x00\x00\x98\x1bZ\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xe8"c,\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xf6\xccd\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80$?b\xac\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x10ld\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80h\xb7a\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x000\xb5a\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x91S\xe60\x85A\xe4\xb0\x12\x00\x00\x00\x12\x00\x00\x80\xc5\xb3\xe40<?\xe50\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00H(\xe50\x12\x00\x00\x00\x12\x00\x00\x00\x0e\x9d\xdd\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xcb{\xe60\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00b|\xe8\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80F\xa1\xe5\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xc3\xaa\xe7\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x96,\xe5\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xd9%\xe5\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80]o\xe5+>U\xe3\xab\x12\x00\x00\x00\x12\x00\x00\x80\xbft\xe3+ZM\xe4+\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00gE\xe4+\x12\x00\x00\x00\x12\x00\x00\x00\x03y\xdc\xab\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80a\x99\xe5+\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x0fH\xe7\xab\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80(\xb3\xe4\xab\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x1d\xe6\xe6\xab\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xf4)\xe4\xab\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xb7\'\xe4\xab\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x9fA\xd8\xb1\xc2c\xd61\x12\x00\x00\x80\x12\x00\x00\x00\x12N\xd6\xb1\xb98\xd7\xb1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xcb?\xd7\xb1\x12\x00\x00\x80\x12\x00\x00\x80Y\x12\xd01\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xc2g\xd8\xb1\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x18\xbb\xd91\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00S\x95\xd71\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80A\xa5\xd91\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x14\x10\xd71\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x89\x10\xd71\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xd0mx,\x0c.v\xac\x12\x00\x00\x00\x12\x00\x00\x80\xc9vv,V@w,\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00B)w,\x12\x00\x00\x00\x12\x00\x00\x00\x8d\xf5n\xac\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x80\x99x,\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80+uz\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xa0\xaaw\xac\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xff\xe5y\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xc1 w\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xd2\x1cw\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80&\\},\xe7\t{\xac\x12\x00\x00\x00\x12\x00\x00\x80\xb2,{,\xe2\x1b|,\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x1d\x13|,\x12\x00\x00\x00\x12\x00\x00\x00\x94vs\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x8b\x8a},\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00&f\x7f\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80T\x8c|\xac\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xf8\xf9~\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xce\xf4{\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00R\xf2{\xac\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\r\xf4\xd5\xaf\xa4\xb4\xd3/\x12\x00\x00\x80\x12\x00\x00\x80P\xd7\xd5\xafb\xe7\xd4\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x9c\x96\xd4\xaf\x12\x00\x00\x00\x12\x00\x00\x00\x15D\xcd/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xf1\x1c\xd6\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80^D\xda/\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00iK\xd5/\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\'\x0f\xd7/\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x002)\xd5/\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x803\t\xd5/\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x806\xbd\x83.q\xe3\x81\xae\x12\x00\x00\x00\x12\x00\x00\x80;E\x82.\xa4\xee\x82.\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x802\x8a\x82.\x12\x00\x00\x00\x12\x00\x00\x00\xae\x7fz\xae\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80c\xdb\x83.\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xcc\xcc\x84\xae\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x826\x83\xae\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80Xk\x84\xae\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xf0\xd1\x82\xae\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x81\xd1\x82\xae\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00^\x10\xee\xaf?\xab\xeb/\x12\x00\x00\x80\x12\x00\x00\x00;O\xec\xaf\t\xac\xec\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xa4\xce\xec\xaf\x12\x00\x00\x80\x12\x00\x00\x80E\x8e\xe3/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x1fE\xee\xaf\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80^T\xf1/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xaf*\xed/\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x0b\r\xf0/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xd3\xa5\xec/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xdd\x97\xec/\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80=O+1\x89\xb6)\xb1\x12\x00\x00\x00\x12\x00\x00\x80`\r*1c\x82*1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x8ba*1\x12\x00\x00\x00\x12\x00\x00\x00l\xc9$\xb1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xffl+1\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00f\xd0,\xb1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xed\xca*\xb1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x07>,\xb1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xc1q*\xb1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x9cm*\xb1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xaa\x02\x9e0\x0ft\x9c\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x915\x9d0\xd6?\x9d0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xd8\x16\x9d0\x12\x00\x00\x00\x12\x00\x00\x00\xfd\xca\x97\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80v\x1f\x9e0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x93\n\xa0\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x0b\x86\x9d\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00|\xdd\x9e\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80sF\x9d\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xb5;\x9d\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xdb\x15\x01\xb0d\xbb\xff/\x12\x00\x00\x80\x12\x00\x00\x00o\xb9\xff\xaf\xd9o\x00\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xaaf\x00\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x01\xee\xf7/\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xcd-\x01\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xba\xff\x010\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xce\xa9\x000\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80s\xe6\x010\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00yV\x000\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xbfV\x000\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x15\xd7\x15\xb0\x0eP\x140\x12\x00\x00\x80\x12\x00\x00\x802\\\x14\xb0\x9a\x10\x15\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x008\xf3\x14\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x98\xae\x0f0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xc3\xf3\x15\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xe9\xdf\x160\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xc7U\x150\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xce\xbe\x160\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00G\xf1\x140\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xec\xf1\x140\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xad\xe2\xc7\xaf-\x15\xc6/\x12\x00\x00\x80\x12\x00\x00\x00:\x0c\xc6\xafd\xe8\xc6\xaf\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80b\xe4\xc6\xaf\x12\x00\x00\x00\x12\x00\x00\x00!(\xc0/\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xc0\x06\xc8\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12J\xc9/\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xdd?\xc7/\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x92\'\xc9/\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x84\xc2\xc6/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xcc\xc2\xc6/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xdd\xd0}\xb0\xb3\x86{0\x12\x00\x00\x80\x12\x00\x00\x00\x18\xac{\xb0!\x91|\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xdc\x8f|\xb0\x12\x00\x00\x80\x12\x00\x00\x80\xc5\xf2s0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x006\xff}\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80#\xe3\x7f0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x7f\x01}0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xd8v\x7f0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x13l|0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x03i|0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xda|\x10.\xf7)\x0f\xae\x12\x00\x00\x80\x12\x00\x00\x00\xcc=\x0f.4\xc6\x0f.\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x004\xc1\x0f.\x12\x00\x00\x80\x12\x00\x00\x80\xfd\xd7\n\xae\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00M\x97\x10.\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x89\xa6\x11\xae\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80R\x06\x10\xae\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xdah\x11\xae\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xe9\xaf\x0f\xae\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x80\xae\x0f\xae\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x94v\x12\xafk\x9c\x10/\x12\x00\x00\x80\x12\x00\x00\x00\x17\xef\x14\xaf\xb2\xc9\x11\xaf\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xb83\x11\xaf\x12\x00\x00\x80\x12\x00\x00\x00eH\x0c/\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xc5\x93\x12\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x003\x1e\x19/\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x98\x12\x12/\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xff\xda\x12/\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xe0\x85\x12/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x00E\x12/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x001\xe1\xfe\xb1h\x99\xfc1\x12\x00\x00\x80\x12\x00\x00\x00\x04\x83\xfc\xb12\xa0\xfd\xb1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xff\xa2\xfd\xb1\x12\x00\x00\x80\x12\x00\x00\x80)\x02\xf51\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00{\x0f\xff\xb1\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xdaT\x002\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00P\x10\xfe1\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80 E\x002\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xdfn\xfd1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80yo\xfd1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x1bN\xa4,\xbd\xcc\xa2\xac\x12\x00\x00\x00\x12\x00\x00\x80N\xe3\xa2,h~\xa3,\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xb7x\xa3,\x12\x00\x00\x00\x12\x00\x00\x00\x11\xe3\x9d\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x802l\xa4,\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x9d\xa0\xa5\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80T\xc7\xa3\xac\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00yZ\xa5\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x0fe\xa3\xac\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00vc\xa3\xac'
param1 = np.fromstring(ts,dtype=theano.config.floatX)

In [30]:
param1.reshape(110,80).shape

(110, 80)

In [32]:
loaded_param[-3].tostring()

'\x12\x00\x00\x00\xd2\x94\xc82\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xc5\xe5v(\x13\x99\x830\x12\x00\x00\x00\x12\x00\x00\x00n\xfc\xde2\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xc2\xc7R:J\x05B,\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80g\x92S,\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00KiT?\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x87\xe8\x11<\xf2\'\xa20u\xb6\x90/\x00\xf3\x813\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00%\xb4\x880\x12\x00\x00\x80nD\x8d*\x93>V4\x12\x00\x00\x00\x12\x00\x00\x00\xc3?(-\x04\x8bT7v\x85Z.\x12\x00\x00\x00#\x19T.\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xfe\x88\x02$=\'\x0b,\x12\x00\x00\x00\x12\x00\x00\x00\xd9\xc9k.\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80[\xe2\xde5\xe1(\xcd\'\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xe7\xb7\xdf\'\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\

In [33]:
tsm3='\x12\x00\x00\x00\xd2\x94\xc82\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xc5\xe5v(\x13\x99\x830\x12\x00\x00\x00\x12\x00\x00\x00n\xfc\xde2\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xc2\xc7R:J\x05B,\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80g\x92S,\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00KiT?\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x87\xe8\x11<\xf2\'\xa20u\xb6\x90/\x00\xf3\x813\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00%\xb4\x880\x12\x00\x00\x80nD\x8d*\x93>V4\x12\x00\x00\x00\x12\x00\x00\x00\xc3?(-\x04\x8bT7v\x85Z.\x12\x00\x00\x00#\x19T.\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xfe\x88\x02$=\'\x0b,\x12\x00\x00\x00\x12\x00\x00\x00\xd9\xc9k.\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80[\xe2\xde5\xe1(\xcd\'\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xe7\xb7\xdf\'\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80(v\xa8:\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x8bI\x9a7\x9bj\xab,d\x05\x19+!i\t/\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x92\x03\x13,\x12\x00\x00\x00\x83`\x15&\xb7\x8b\xe2/\x12\x00\x00\x80\x12\x00\x00\x80i\xe8\xb1(h\xbf\xe02\x1b\x11\xe7)\x12\x00\x00\x00|\xfbU1\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80&\xb2\x03\'\xd4c\x0c/\x12\x00\x00\x00\x12\x00\x00\x00O\xe2m1\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x08\xdd\xe08\xb9\xfb\xce*\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xfc\xb4\xe1*\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x0e\xf5\xa9=\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00<\xa8\x9b:T\xfd,/\x7fa\x1a.\xaa\xa1\n2\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00J\xd6\x11/\x12\x00\x00\x80a\xb4\x16)\t\x8f\xe42\x12\x00\x00\x00\x12\x00\x00\x808}\xb3+t\xbe\xe25\xce\x1e\xe9,\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x802\x9f(2\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xc4\x8e\xcf\'gB\xdd/\x12\x00\x00\x80\x12\x00\x00\x80\xcet;2\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80E2\xb19%\x1b\xa3+\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x86\xdc\xb1+\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xebC\x84>\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80,Ru;\xa7Q\x080PO\xf3.\xe8|\xda2\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80*\xd8\xe5/\x12\x00\x00\x00)\x84\xed)\xaa\x1b\xb43\x12\x00\x00\x80\x12\x00\x00\x80\xccp\x8d,\x92\xad\xb26\x07\xb4\xb7-\x12\x00\x00\x80z\xdc\x942\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00n:7(\xd1SC0\x12\x00\x00\x80\x12\x00\x00\x80H|\xa52\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80Dn\x1c:7\xfd\x0f,\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x98\x03\x1d,\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80U\xaa\x9d>\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xad\x92\xd8;\x0f\xad\xf00\xf2\xcaV/g\xe1@3\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xd4-K0\x12\x00\x00\x00\x1c\xadQ*\x15\x00\x1f4\x12\x00\x00\x80\x12\x00\x00\x00\xd9\xb9\xf9,\x03\xbd\x1d7\x13,".\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xe5~\x10\xb3\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xa9\xd8\xb1\xa8\x9c\x97\xbd\xb0\x12\x00\x00\x80\x12\x00\x00\x80\xe0\x9f \xb3\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80"\xd7\x97\xba\xd9\xc2\x8b\xac\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00Cg\x98\xac\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xbe\x19\x19\xbf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xee7R\xbc\x89\x97i\xb1\x81|\xd0\xaf\x1c7\xbb\xb3\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80f\xf2\xc5\xb0\x12\x00\x00\x00\xc4\x84\xcb\xaamU\x9a\xb4\x12\x00\x00\x80\x12\x00\x00\x80sdr\xad\xf2\x1b\x99\xb7\x1di\x9d\xae\x12\x00\x00\x00\xbft\x9b/\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00YZ?%\xe6\xfbK-\x12\x00\x00\x80\x12\x00\x00\x00\xf5\xd1\xac/\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x1b\\#7\x1e_\x16)\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x96\xf9#)\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xa7\x9f$<\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x804*\xe28PY\xfb-\x00P`,\xa4mI0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\xc9\xa1-\x12\x00\x00\x80\xe4\xf8Z\'\x91\x0b&1\x12\x00\x00\x00\x12\x00\x00\x00\xbbe\x02*\xeb\xb9$47\\)+\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\\n\x053\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xc3<\xa4(\x0e\x15\xaf0\x12\x00\x00\x00\x12\x00\x00\x00\xadU\x143\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xb77\x8c:\xa3\x10\x81,\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80~\xbd\x8c,\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xf9M\x8d?\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80j B<\x0f\xbd\xd70\xc8\x87\xc0/\xf7\xe3\xac3\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00@\xe0\xb50\x12\x00\x00\x80\xc6\xf1\xbb*1\x85\x8e4\x12\x00\x00\x00\x12\x00\x00\x00\xa0\xd7_-\xbbc\x8d7\x14]\x91.\x12\x00\x00\x00"\xcf\xec3\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xcc\xbd\x91)\x8a]\x9b1\x12\x00\x00\x00\x12\x00\x00\x00O\xa1\x034\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x98\xdax;\x86\x0fe-\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x00\xc8y-\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x83\xc8\xfa?\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xd2C,=Yq\xbf15\xd9\xaa0\x86k\x994\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xc6d\xa11\x12\x00\x00\x80d\xc7\xa6+\xd9\xf0|5\x12\x00\x00\x00\x12\x00\x00\x00O\xa2F.)\xefz8N\xfe\x80/\x12\x00\x00\x80>\x98\x84\x15\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xafxI\x0b\xa4\x00\x8f\x13\x12\x00\x00\x00\x12\x00\x00\x80\xf1\xde\x92\x15\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00I\x8c \x1d\xf4\x16E\x0f\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\r"C\x0f\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xf1e{"\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x005\x13\xf7\x1e\x14\x01\xd6\x13\t\xf7\xa1\x12.\xc7-\x16\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x804o\xb3\x13\x12\x00\x00\x00\xc8\x1c\xb7\rTs\x11\x17\x12\x00\x00\x80\x12\x00\x00\x00\xa4\xc9!\x10A\xcb\x1a\x1a\xcd^\x8f\x11\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xc6{\xeb0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xce\xec\x90&\xd6~\x9a.\x12\x00\x00\x00\x12\x00\x00\x00\x9c\xe4\x021\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00(vw8H\xc7c*\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x0cbx*\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00oay=\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\tM+:\xf7^\xbe.f\xe4\xa9-\xc1\x8f\x981\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x84}\xa0.\x12\x00\x00\x00!\xd8\xa5(\x91\x86{2\x12\x00\x00\x00\x12\x00\x00\x00\x9b\x85E+\x9f\x87y5ZE\x80,\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80?Sv\x95\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xdf\x10;\x8bm\xef\x84\x93\x12\x00\x00\x00\x12\x00\x00\x00qf\x88\x95\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xae]\x15\x9d{k7\x8f\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\xa45\x8f\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x9a\x83j\xa2\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x07Q\xe6\x9e\xc5O\xc7\x93\xa4\xa8\x96\x92&\xaa!\x96\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xa3\xe4\xa6\x93\x12\x00\x00\x00\xd3Z\xaa\x8d;\x91\x07\x97\x12\x00\x00\x80\x12\x00\x00\x808\x95\x16\x90)\xac\x0f\x9a\xc9\xd9\x85\x91\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xd7\xc9 \xb3\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xb2\xe8\xc5\xa8\xbb\xfa\xd2\xb0\x12\x00\x00\x00\x12\x00\x00\x00p\xbf2\xb3\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xae\xf7\xa8\xba\xe3\x86\x9b\xac\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x80\x98\xa9\xac\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00@G\xaa\xbf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00#\xeei\xbcF\xfc\x01\xb1(\x01\xe8\xaf\xa1V\xd0\xb3\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00l*\xdb\xb0\x12\x00\x00\x80.z\xe2\xaa\xdb\xbd\xab\xb4\x12\x00\x00\x00\x12\x00\x00\x00<\xde\x86\xad/a\xaa\xb7\xbe*\xaf\xae\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xe2H\xf22\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xe1\x1b\x95(\x1a\xf4\x9e0\x12\x00\x00\x80\x12\x00\x00\x80\xcb\xaa\x063\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x806\x99~:\xffYj,\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xff\x8c\x7f,\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80GU\x00?\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xa5=0<I\xd6C1]\xcb\xae/\xca\xf5\x9c3\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x8e\xb5\xa50\x12\x00\x00\x00\xc9\xa1\xaa*\x10d\x814\x12\x00\x00\x80\x12\x00\x00\x80\xe98K-\x08]\x807\x04\xf9\x83.\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80I\xabZ\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x9e\x93\x06\xa6\xc1v\x0f\xae\x12\x00\x00\x80\x12\x00\x00\x80\x0b\x18s\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xd8\xca\xe5\xb7\x9c\x83\xd3\xa9\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00~\xa5\xe6\xa9\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x07\xaf\xad\xbc\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xef\x11\x9f\xb9!]\xb1\xae\xcc\xc2\x1d\xad\r\xab\r\xb1\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xe4\x82\x95\xae\x12\x00\x00\x00\xa5\x00\x1a\xa8\xc4\x90\xe9\xb1\x12\x00\x00\x80\x12\x00\x00\x80\xf9j\xb7\xaap\xb6\xe7\xb4\\9\xee\xab\x12\x00\x00\x80\x94\x06\xcc\xb3\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xdf {\xa9=\xdb\x85\xb1\x12\x00\x00\x80\x12\x00\x00\x80\x97\xd0\xe2\xb3\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xa9gV\xbbyYE\xad\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00f3W\xad\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80p\x11\xd8\xbf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x10k\x14\xbd\x8e\xf0\xa4\xb1:2\x93\xb0g.\x84\xb4\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xf0\x0c\x8b\xb1\x12\x00\x00\x00m\xb0\x8f\xab\xb0\xecY\xb5\x12\x00\x00\x80\x12\x00\x00\x80}"+\xae82X\xb8KE^\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xd9\xd3\x163\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x15\xa7\xb9(\xdc\xe8\xc50\x12\x00\x00\x00\x12\x00\x00\x00\xc1\xac\'3\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x80\x7f\x9e:\xa4\xe4\x91,\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00>\x17\x9f,\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xc3\xb9\x9f?\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00+o[<\xd5\xdd\xf30\r\xa2\xd9/\x81n\xc33\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xc6\x96\xcd0\x12\x00\x00\x00\x17s\xd4*\x14\x1a\xa14\x12\x00\x00\x00\x12\x00\x00\x002\x07}-\xa8\xd2\x9f7\xfcP\xa4.\x12\x00\x00\x00[\x9fz\xb0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x15>\x1a\xa6\xa1m$\xae\x12\x00\x00\x00\x12\x00\x00\x80\xe6N\x8b\xb0\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80v\xaf\x03\xb8\x1el\xf2\xa9\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xe9,\x04\xaa\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xca\x14\xc7\xbc\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80bP\xb6\xb9.\x9cJ\xaeo\xd04\xad\x98^"\xb1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xe7\xce*\xae\x12\x00\x00\x00\xc2\x810\xa8\x00\xd9\x05\xb2\x12\x00\x00\x80\x12\x00\x00\x00n8\xd2\xaa.\xc9\x04\xb5\x92\x84\x08\xac\x12\x00\x00\x00\xf0\xf9P2\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xdb\x9c\x00(\xde\x1a\t0\x12\x00\x00\x00\x12\x00\x00\x00uQh2\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xc7\x9a\xdb9\xbf#\xca+\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xb7l\xdc+\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00?\xe9\xa5>\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00p\x04\x98; \xf1(0\xbe\xc4\x16/[c\x073\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xbdl\x0e0\x12\x00\x00\x80W-\x13*26\xdf3\x12\x00\x00\x00\x12\x00\x00\x00\xecI\xaf,\xc4p\xdd6A\xaa\xe3-\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00$\xc8\x821\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00~\xf9 \'\x05\x9b+/\x12\x00\x00\x00\x12\x00\x00\x00\xd1c\x911\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00Wo\t9\xfd\x00\xfd*\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xe6\xf1\t+\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x8d\x80\n>\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x806F\xbe:\xd8q\xd3/\x05\xb5<.<u)2\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x9cA\xb2/\x12\x00\x00\x80&68)\xf6\xb0\x0b3\x12\x00\x00\x00\x12\x00\x00\x00\x97e\xdb+Y\x95\n6\x08z\x0e-\x12\x00\x00\x00L?\xbf\xaf\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xb4fk\xa51\xf2z\xad\x12\x00\x00\x80\x12\x00\x00\x00\xe1\x9b\xd4\xaf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xab\xf9H\xb7;\xfd8\xa9\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80,\xb9I\xa9\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00M\xe9\x17\xbc\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80.\x1f\x0b\xb9\xea\x9b\x9a\xad&\xfa\x89\xac\xf3\xcdw\xb0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80xW\x82\xad\x12\x00\x00\x80\xb3\xb0\x86\xa7\x81FL\xb1\x12\x00\x00\x00\x12\x00\x00\x00\xa6j \xaa\xb1\xa7J\xb4\x85YP\xab\x12\x00\x00\x807\xe7u\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80+W\x17\xa6\x19U!\xae\x12\x00\x00\x00\x12\x00\x00\x80V\xaf\x88\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00<4\x01\xb8\xec\xdb\xed\xa9\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x1d\xb0\x01\xaa\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xe5R\xc3\xbc\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xbf\xe0\xb2\xb9\x90\x99\xc5\xae\ni1\xad\xcbO\x1f\xb1\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80^\x95\xa6\xae\x12\x00\x00\x80i/-\xa8\xa4S\x03\xb2\x12\x00\x00\x80\x12\x00\x00\x80\x81C\xce\xaa\xccH\x02\xb5\xa5\xf2\x05\xac\x12\x00\x00\x80h\xf9Y\xb0\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00b\'\x06\xa6S\x02\x0f\xae\x12\x00\x00\x80\x12\x00\x00\x807Rr\xb0\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80v\x0e\xe5\xb7H\xd8\xd2\xa9\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x003\xeb\xe5\xa9\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80/\x1f\xad\xbc\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x004\x8f\x9e\xb9\x8670\xae\xceB\x1d\xad\x957\r\xb1\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\xc9\x8e\x14\xae\x12\x00\x00\x00Q\x84\x19\xa8H\xd2\xe8\xb1\x12\x00\x00\x80\x12\x00\x00\x80\x9a\xd6\xb6\xaa\xe5\xf8\xe6\xb4Nx\xed\xab\x12\x00\x00\x80{\x15\\2\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00|r\x07(Pd\x100\x12\x00\x00\x80\x12\x00\x00\x80\x98\xaat2\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80YG\xe79\xd9\xe1\xd4+\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xaa#\xe8+\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80{\x12\xe9>\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00P\x19\xa0;\xd2\xeb10\x07\xc8\x1e/\x9c\x95\x0e3\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x98\xfe\x150\x12\x00\x00\x00\xb0\xff\x1a*s\x13\xeb3\x12\x00\x00\x80\x12\x00\x00\x80\xbc\x9a\xb8,\x176\xe96\xbd\xc3\xef-\x12\x00\x00\x00\xde{<0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80F\x02\xe8%^R\xf7-\x12\x00\x00\x00\x12\x00\x00\x00G\x89Q0\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xd6\x10\xc67\xb9Q\xb6)\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xf4\xcf\xc6)\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xe8\x98\xc7<\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00G\x1b\x899:`\x18.\x18\xfc\x07-\x059\xf40\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00vu\x00.\x12\x00\x00\x00s\xbf\x04(rR\xc91\x12\x00\x00\x00\x12\x00\x00\x00\x1e\x1a\x9e*\x07\xb9\xc74\x89W\xcd+\x12\x00\x00\x80\xcc\x9e\x153\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xda&\xb8(yS\xc40\x12\x00\x00\x80\x12\x00\x00\x80)U&3\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xb6:\x9d:\xc6\xb9\x90,\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00I\xd1\x9d,\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x8f\x83v?\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xbf\xadY<e\x98r1$\xe4\xd7/\x0f\xde\xc13\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80p\x84L1\x12\x00\x00\x00\xea\xbf\xd2*\xfe\xcf\x9f4\x12\x00\x00\x80\x12\x00\x00\x80\xa8\x00{-B\x8b\x9e7f\x00\xa3.\x12\x00\x00\x00Y\t\x123\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xf6\xbc\xb3(@\x9f\xbf0\x12\x00\x00\x00\x12\x00\x00\x00(Y"3\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x16w\x99:\xf4A\x8d,\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x0c\t\x9a,\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xa6\xfbg?\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xb8wT<\xa2-l1\xe3\xb7\xd2/W9\xbd3\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x97\x1bG1\x12\x00\x00\x80\xe4\xb2\xcd*\x0c\xfc\x9b4\x12\x00\x00\x00\x12\x00\x00\x00\xdb\xfct-i\xbf\x9a7}\x18\x9f.\x12\x00\x00\x00\xacTR2\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00?r\x01(=\xfe\t0\x12\x00\x00\x00\x12\x00\x00\x00\x82\xd2i2\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xd7\x06\xdd9\x02s\xcb+\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80=\xda\xdd+\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00I\xc1^>\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00n\x00\x99;\x81\xec\xa90\xc5\xbe\x17/\xc9C\x083\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x86q\x0f0\x12\x00\x00\x80\x89!\x14*]\xa8\xe03\x12\x00\x00\x00\x12\x00\x00\x00\xa2l\xb0,\xe4\xdf\xde6\xd3#\xe5-\x12\x00\x00\x80\t\xe5\xd3.\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\xe9h\x82$\x0e\x05\x8b,\x12\x00\x00\x00\x12\x00\x00\x80\xfa\x8f\xeb.\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xab\xab^6l\xf6L(\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xf1\x80_(\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80 e\xe0:\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xa3#\x1a8+M\xab,\xd1\xdf\x98+lG\x89/\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x04j\x90,\x12\x00\x00\x00\xd3;\x95&0Tb0\x12\x00\x00\x80\x12\x00\x00\x80\xcf\xbc1)D\x88`3`\xd8f*\x12\x00\x00\x00[\xf1~3\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x86\xe8\x1c)\x8eC\'1\x12\x00\x00\x00\x12\x00\x00\x00\x9f\xb5\x8d3\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80m\xf3\x05;\x03\x9b\xf6,\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80)u\x06-\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x13\xf8\xd9?\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xd9r\xb9<\x8c\x1aN1\xe0\xee70\xec*%4\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xed\xc0-1\x12\x00\x00\x00=\x8e3+W\'\x085\x12\x00\x00\x00\x12\x00\x00\x00u\xd9\xd5-r\x12\x078\x9e\xdf\n/\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00h\xc3#3\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xc3\x92\xc9(3\xe2\xd60\x12\x00\x00\x80\x12\x00\x00\x80&\x0e63\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\xb4\x17\xac:\xc3g\x9e,\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x1c\xbc\xac,\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xf6\xe9\x84?\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x82An<\x06d\x041ML\xec/x1\xd43\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\xaf8\xdf0\x12\x00\x00\x80N\xab\xe6*X\xeb\xae4\x12\x00\x00\x80\x12\x00\x00\x00*]\x89-\xf6\x87\xad7\xa1h\xb2.\x12\x00\x00\x00e3\x0e3\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xd3\t\xaf(H\x97\xba0\x12\x00\x00\x00\x12\x00\x00\x00\x98\x15\x1e3\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\tn\x95:\xe1\x8c\x89,\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80a\xfe\x95,\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00S\x9ae?\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xe9\xe0N<L\xeb\xe50\xb8/\xcd/\xd8@\xb83\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\xbe\xd4\xc10\x12\x00\x00\x80\x8cM\xc8*\t\xe3\x974\x12\x00\x00\x00\x12\x00\x00\x008\x8fn-\x15\xae\x967\x84\xeb\x9a.\x12\x00\x00\x00\x82s\x8c2\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00:\xe2,(rK80\x12\x00\x00\x00\x12\x00\x00\x00\xa1#\x9c2\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80z\x97\x13:\x89\xdb\x07,\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\xcb%\x14,\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xae\x1e\xdf>\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\xa7U\xcc;\x05\x17c0r\xa9J/|\xfc53\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80(r?0\x12\x00\x00\x80B\xd6E*\x9b\x04\x164\x12\x00\x00\x80\x12\x00\x00\x00\xa5\x9f\xeb,\x94\xd3\x147g\x03\x19.\x12\x00\x00\x80\xafb\x18\xb3\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x80A\x92\xbb\xa8b\xf4\xc7\xb0\x12\x00\x00\x00\x12\x00\x00\x80!h)\xb3\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80p"\xa0\xba\x87f\x93\xac\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x1e\xbc\xa0\xac\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\xd5_\xa1\xbf\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x00\x12\x00\x00\x809\xb3]\xbc\xbfb\xf6\xb0\xa6\xe1\xdb\xafAs\xc5\xb3\x12\x00\x00\x00\x12\x00\x00\x80\x12\x00\x00\x80\x80\xb6\xcf\xb0\x12\x00\x00\x00/\xa5\xd6\xaa\x12\xc4\xa2\xb4\x12\x00\x00\x00\x12\x00\x00\x00u\xa4\x7f\xad5y\xa1\xb7\x9f\x03\xa6\xae'
paramm3 = np.fromstring(tsm3,theano.config.floatX)

In [37]:
print(paramm3.shape)
print(paramm3.reshape((50,50)).shape)
paramm3.reshape((50,50))

(2500,)
(50, 50)


array([[  2.52233724e-44,   2.33507400e-08,   2.52233724e-44, ...,
          9.56385231e-12,   1.26685518e-05,   4.96859984e-11],
       [  2.52233724e-44,   4.82255451e-11,   2.52233724e-44, ...,
          1.97517394e-14,   2.61640736e-08,   1.02614281e-13],
       [  2.52233724e-44,   3.11385318e-09,   2.52233724e-44, ...,
          1.27534702e-12,   1.68937413e-06,   6.62567830e-12],
       ..., 
       [  2.52233724e-44,   3.31086945e-08,   2.52233724e-44, ...,
          1.35605346e-11,   1.79624567e-05,   7.04494518e-11],
       [  2.52233724e-44,   1.63506719e-08,   2.52233724e-44, ...,
          6.69682582e-12,   8.87074930e-06,   3.47911942e-11],
       [ -2.52233724e-44,  -3.54800100e-08,   2.52233724e-44, ...,
         -1.45315886e-11,  -1.92491370e-05,  -7.54944926e-11]], dtype=float32)

In [27]:
Memblck.__get_state__()[2]

Total number of parameters: 19 


array([[  2.52233724e-44,  -2.52233724e-44,  -2.52233724e-44, ...,
          2.52233724e-44,   2.52233724e-44,  -3.80828835e-09],
       [  2.52233724e-44,   2.52233724e-44,   2.52233724e-44, ...,
         -2.52233724e-44,   2.52233724e-44,  -3.80828835e-09],
       [  2.52233724e-44,  -2.52233724e-44,   2.52233724e-44, ...,
          2.52233724e-44,   2.52233724e-44,  -3.80828835e-09],
       ..., 
       [  2.52233724e-44,   2.52233724e-44,   2.52233724e-44, ...,
         -2.52233724e-44,  -2.52233724e-44,  -3.80828835e-09],
       [  2.52233724e-44,   2.52233724e-44,  -2.52233724e-44, ...,
         -2.52233724e-44,  -2.52233724e-44,  -3.80828835e-09],
       [  2.52233724e-44,  -2.52233724e-44,  -2.52233724e-44, ...,
          2.52233724e-44,   2.52233724e-44,   3.80828835e-09]], dtype=float32)

In [51]:
[1,2,3]

[1, 2, 3]

In [53]:
pd.Dataframe

AttributeError: 'module' object has no attribute 'Dataframe'